In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
file_path = "dataset/cicdarknet.parquet"  # Modify based on actual file location
df = pd.read_parquet(file_path)

# Keep only relevant columns
X = df.drop(columns=["Label", "Label.1"], errors='ignore')  # Removing unnecessary labels
y = df["Label"].apply(lambda x: 1 if x == "Tor" else 0)  # Convert to binary classification

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))


(103121,)